In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [2]:
URL = "https://www.offiziellecharts.de/charts/single/for-date-695867602000"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [6]:
url_base = "https://www.offiziellecharts.de"
url_start = "/charts/single/for-date-315529200000"
url_end = "/charts/single/for-date-630975600000"
url_next = 315529200000
url_end2 = 638575200000
data_lilst = []
#finish with the last week of 1989
while url_next < url_end2:
    page = requests.get(url_base + url_start)
    soup = BeautifulSoup(page.content, 'html.parser')
    #get link for next week
    soup_next_url = soup.find("a", class_="next-link btn btn-default btn-sm")
    url_start = soup_next_url["href"]
    url_next = int(re.findall("\d+", url_start)[0])
    print(url_next)
    soup_chart = soup.find("table", class_="table chart-table")
    soup_chart = soup_chart.find_all("tr", class_="drill-down-link")
    for soup_song in soup_chart:
        soup_song = soup_song.find("td", class_="ch-info")
        soup_song = soup_song.find("div", class_="wrap")
        artist = soup_song.find("span", class_="info-artist").text.strip()
        title = soup_song.find("span", class_="info-title").text.strip()
        label = soup_song.find("span", class_="info-label").text.strip()
        soup_song_data = soup_song.find("span", class_="play-media")
        soup_song_data = soup_song_data.find_all("span", class_="plus-data")
        for soup_song_data in soup_song_data:
            if "Peak:" in soup_song_data.text.strip():
                peak = soup_song_data.find("b").text.strip()
                peak = int(peak)
            else:
                weeks = soup_song_data.find("b").text.strip()
                weeks = re.findall("\d+", weeks)[0]
        #add to data list
        data_lilst.append([artist, title, label, peak, weeks])
        #time.sleep(1)
df = pd.DataFrame(data_lilst, columns=["Artist", "Title", "Label", "Peak", "Weeks"])
df

316109498000
316714299000
317319099000
317923899000
318528700000
319133500000
319738300000
320343101000
320947901000
321552701000
322157502000
322762302000
323367102000
323968303000
324573103000
325177903000
325782704000
326387504000
326992305000
327597105000
328201905000
328806706000
329411506000
330016306000
330621108000
331225908000
331830708000
332435509000
333040309000
333645110000
334249910000
334854710000
335459511000
336064311000
336669111000
337273912000
337878712000
338483512000
339091913000
339696713000
340301514000
340906314000
341511114000
342115915000
342720715000
343325515000
343930316000
344535116000
345139916000
345744717000
346349517000
346954317000
347559118000
348163918000
348768718000
349373519000
349978319000
350583119000
351187920000
351792720000
352397521000
353002321000
353607121000
354211922000
354813122000
355417922000
356022723000
356627523000
357232324000
357837124000
358441924000
359046725000
359651525000
360256325000
360861125000
361465926000
362070726000

,Artist,Title,Label,Peak,Weeks
0,Thom Pace,Maybe,RSO,1,11
1,Buggles,Video Killed The Radio Star,Island,2,8
2,Howard Carpendale,"Nachts, wenn alles schläft",EMI,3,14
3,Goombay Dance Band,Sun Of Jamaica,CBS,4,4
4,ABBA,Gimme! Gimme! Gimme! (A Man After Midnight),Polydor,3,10
...,...,...,...,...,...
40445,Jimmy Somerville feat. June Miles Kingston,Comment te dire adieu,London-Metronome,25,14
40446,Lisa Stansfield,This Is The Right Time,Arista,17,25
40447,Double Trouble & Rebel MC,Street Tuff,Metronome,12,18
40448,Stevie B.,The Stevie B. Megamix,BCM,99,1


In [7]:
#search for abba in the dataframe
df[df["Artist"] == "ABBA"]

,Artist,Title,Label,Peak,Weeks
4,ABBA,Gimme! Gimme! Gimme! (A Man After Midnight),Polydor,3,10
38,ABBA,I Have A Dream,Polydor,39,2
52,ABBA,Gimme! Gimme! Gimme! (A Man After Midnight),Polydor,3,11
53,ABBA,I Have A Dream,Polydor,4,3
98,ABBA,Voulez-vous,Polydor,14,15
...,...,...,...,...,...
12166,ABBA,Under Attack,Polydor,22,7
12170,ABBA,The Day Before You Came,Polydor,5,15
12250,ABBA,Under Attack,Polydor,22,8
12270,ABBA,The Day Before You Came,Polydor,5,16


In [8]:
#delete duplicates with the same artist and title and lowest weeks
df = df.sort_values(by=["Artist", "Title", "Weeks", "Peak"])
df = df.drop_duplicates(subset=["Artist", "Title"], keep="first")
df

,Artist,Title,Label,Peak,Weeks
28442,16 Bit,Changing Minds,Ariola,43,1
26649,16 Bit,Where Are You?,Ariola,50,1
114,2 Plus 1,"Easy Come, Easy Go",Autobahn,40,3
39796,49ers,Touch Me,BCM,47,1
25004,5 Star,Can't Wait Another Minute,RCA,55,1
...,...,...,...,...,...
9723,À La Carte,In The Summer Sun Of Greece,Coconut,74,1
3471,À La Carte,"Ring Me, Honey",Hansa Int.,47,1
6182,À La Carte,River Blue,Coconut,58,1
8328,À La Carte,Viva Torero,Coconut,29,1


In [9]:
#save dataframe to csv
df.to_csv("charts.csv", index=False)